In [2]:
import dask
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client

import numpy as np
from dask.array.slicing import shuffle_slice
from dask_ml.model_selection import train_test_split
import hvplot.dask
from dask.array import histogram
import hvplot.pandas
import hvplot.dask

seed = 25
client = Client()

In [ ]:
#adding all train, test, and validation datasets together to resplit to provide multiple reviews/reviewer in test set
interactions_train = dd.read_csv("recipes/interactions_train.csv", blocksize = 25e6)
interactions_test = dd.read_csv("recipes/interactions_test.csv", blocksize = 25e6)
interactions_validation = dd.read_csv("recipes/interactions_validation.csv", blocksize = 25e6)

interactions = dd.concat([interactions_train, interactions_test])
interactions = dd.concat([interactions, interactions_validation])

interactions[['rating']] = interactions[['rating']].astype(int)

def permute_data_dd(df):
    d_arr = df.to_dask_array(True)
    df_len = len(df)
    np.random.seed(42)
    index = np.random.choice(df_len, df_len, replace=False)
    d_arr = shuffle_slice(d_arr, index)
    return d_arr.to_dask_dataframe(df.columns)

interactions = permute_data_dd(interactions)

train, test = train_test_split(interactions, test_size=0.20, random_state = seed)

print(len(train))
print(len(test))

print('train mean:', train[['rating']].compute().mean())
print('train sd:', train[['rating']].compute().std())

print('test mean:', test[['rating']].compute().mean())
print('test sd:', test[['rating']].compute().std())

train.compute().hvplot.hist(y='rating', bins=5, bin_range=(0, 5))

In [ ]:
test.compute().hvplot.hist(y='rating', bins=5, bin_range=(0, 5))